# PyZX interoperability: Pauli webs

In [ ]:
import os
import sys
import pyzx as zx

from pyzx.pauliweb import compute_pauli_webs

from topologiq.scripts.runner import runner
from topologiq.scripts.graph_manager import prep_3d_g
from topologiq.utils.interop_pyzx import pyzx_g_to_simple_g
from topologiq.utils.grapher import lattice_to_g, vis_3d_g

repo_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
output_folder_path = f"{repo_root}/output/txt"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

zx.settings.colors = zx.rgb_colors
%matplotlib widget

This notebook offers an example of how perform an algorithmic lattice surgery with ***topologiq*** with an emphasis on transfering Pauli webs available in the original PyZX graph to the final result.

The notebook goes over early parts of the algorithmic lattice surgery relatively quickly. Check other notebooks for insight into early steps of the process.

## Input: PyZX graph

Let's start by preparing a PyZX graph. We'll use a graph similar to those in the [PyZX notebook that introduces Pauli Webs](https://nbviewer.org/github/zxcalc/pyzx/blob/master/demos/PauliWebsRGB.ipynb).

Note that while we draw the graph using `zx.draw()`, we also save the matplotlib figure separately. We'll use it later.

In [ ]:
circuit_name = "pyzx_example_pauli_webs"

c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
cx q[2], q[1];
cx q[0], q[2];
cx q[2], q[1];
""")
pyzx_graph = c.to_graph()

zx.draw(pyzx_graph, labels=True)
fig_data = zx.draw_matplotlib(pyzx_graph, labels=True)

Let's now compute and examine all Pauli Webs in this graph. For context, in PyZX, a Pauli web is generated by reference to the boundary in each qubit line, and there are two webs per boundary, X and Z.

In [ ]:
order, zwebs, xwebs = compute_pauli_webs(pyzx_graph)
zx.draw(pyzx_graph, labels=True, pauli_web=xwebs[11])
zx.draw(pyzx_graph, labels=True, pauli_web=zwebs[11])
zx.draw(pyzx_graph, labels=True, pauli_web=xwebs[12])
zx.draw(pyzx_graph, labels=True, pauli_web=zwebs[12])
zx.draw(pyzx_graph, labels=True, pauli_web=xwebs[13])
zx.draw(pyzx_graph, labels=True, pauli_web=zwebs[13])

## Process: Running the algorithm

To feed the graph into the algorithm, we must convert it into ***topologiq***'s native format, a `simple_graph`: a simple dictionary of nodes and edges. The conversion removes information available in the original PyZX graph, including information related to Pauli webs.

This is momentarily fine. ***Topologiq*** does not need information about Pauli webs to perform its foundational task of placing spiders in the 3D space. 

That said, we will recover that information later in the process.

In [ ]:
simple_graph = pyzx_g_to_simple_g(pyzx_graph)
print(simple_graph)

The next step is to give the `simple_graph` to ***topologiq*** and let it work its magic (it's not magic, but let's pretend it is).

In short, ***topologiq*** will traverse that graph using a Breadth-First Search (BFS) approach and exchange all spiders and edges in the original graph for a 3D version of themselves.

If/when the lattice surgery is complete, ***topologiq*** will produce:
- text results saved to `[root]/outputs/txt/` that includes the final surgery/space-time diagram,
- and related objects for programmatic use.

Additional visualisations options are available but come at a significant cost in runtimes. Check ***topologiq***'s [README](../../README.md) for details.

In [ ]:
# Define weights for the value function to to choose best of several valid paths per each edge based on: (length of path, number of beams broken by path)
# A negative value for length of path favours short paths.
# A negative value for number of beams broken by path favours placements that do not block potential open faces requiring connections.
VALUE_FUNCTION_HYPERPARAMS = (-1, -1)

# Define a desired length of beams
# The longer the beams, the more space between faces. This feature needs improvement but, in theory, increases the odds of success.
# Needs to be combined with an equal or larger `MAX_PATHFINDER_SEARCH_SPACE`
LENGTH_OF_BEAMS = 9

kwargs = {"weights": VALUE_FUNCTION_HYPERPARAMS, "length_of_beams": LENGTH_OF_BEAMS}

simple_graph_after_use, edge_paths, lattice_nodes, lattice_edges = runner(
    simple_graph,
    circuit_name,
    strip_boundaries=False,
    hide_boundaries=False,
    max_attempts=10,
    vis_options=("final", None),
    fig_data=fig_data,  # This is where we use the fig_data from PyZX, so it's overlaid on top of progress and final visualisations
    **kwargs
)

## Output: Rebuilding Pauli webs.

It is possible to rebuild Pauli webs because there is a relation between the spiders and edges in the PyZX graph and the corresponding 3D blocks in the objects created by the algorithm.
- ***Topologiq*** maintains the IDs of any spider in the original graph, using additional/different IDs for intermediate blocks needed to clear any paths.
- ***Topologiq*** also notes the original edge for all 3D edges and edge segments.

The challenge of rebuilding Pauli webs comes down to applying each specific Pauli web to the corresponding 3D blocks.
- Short edges are easy because they are a direct instantiation of the original edge so the original Pauli web half-edge segments apply.
- Long edges are hard because these result from an original edges being broken into several edge segments, which requires applying relations to all edge segments in a given long edge.

But let's start by printing the pure lattice surgery / space-time diagram objects produced by the algorithm. It is always good to inspect results closely before manipulating them.

In [ ]:
if lattice_nodes:
    for key, node in lattice_nodes.items():
        print(f"Node ID {key}: {node}.")

if lattice_edges:
    for key, edge in lattice_edges.items():
        print(f"Edge ID {key}: {edge[0]} (corresponding edge in ZX-graph: {edge[1]}).")

Having familiarised ourselves with the pure objects produced by the algorithm, we can now start transferring information about the original Pauli webs. 

The first step is to get an edge-by-edge summary of each Pauli web in the original PyZX graph.

In [ ]:
boundaries = [key for key, value in pyzx_graph.types().items() if value == 0]
boundaries = boundaries[pyzx_graph.qubit_count() :]

original_pyzx_z_webs_half_edges = []
original_pyzx_x_webs_half_edges = []

for id in boundaries:
    z_edges_temp = {}
    for key, val in zwebs[id].half_edges().items():
        sorted_key = tuple(sorted(key))
        if sorted_key not in list(z_edges_temp.keys()):
            z_edges_temp[sorted_key] = val
    original_pyzx_z_webs_half_edges.append(z_edges_temp)

    x_edges_temp = {}
    for key, val in xwebs[id].half_edges().items():
        sorted_key = tuple(sorted(key))
        if sorted_key not in list(x_edges_temp.keys()):
            x_edges_temp[sorted_key] = val
    original_pyzx_x_webs_half_edges.append(x_edges_temp)


for edge in original_pyzx_z_webs_half_edges:
    print(edge)
print()
for edge in original_pyzx_x_webs_half_edges:
    print(edge)

Next, we go over the extracted webs and determine if ***topologiq*** kept the original edge untouched or had to break it into segments.

In [ ]:
zwebs_3d = []
xwebs_3d = []

if lattice_edges:

    for original_pyzx_z_webs_edge in original_pyzx_z_webs_half_edges:

        original_pyzx_z_webs_edge_keys_sorted = [
            tuple(sorted(key)) for key in original_pyzx_z_webs_edge.keys()
        ]

        current_web = {}
        for key, lattice_edge in lattice_edges.items():
            original_key_for_lattice_edge = tuple(sorted(lattice_edge[1]))
            if original_key_for_lattice_edge in original_pyzx_z_webs_edge_keys_sorted:
                current_web[key] = original_pyzx_z_webs_edge[
                    original_key_for_lattice_edge
                ]
        zwebs_3d.append(current_web)

    for original_pyzx_x_webs_edge in original_pyzx_x_webs_half_edges:

        original_pyzx_x_webs_edge_keys_sorted = [
            tuple(sorted(key)) for key in original_pyzx_x_webs_edge.keys()
        ]

        current_web = {}
        for key, lattice_edge in lattice_edges.items():
            original_key_for_lattice_edge = tuple(sorted(lattice_edge[1]))
            if original_key_for_lattice_edge in original_pyzx_x_webs_edge_keys_sorted:
                current_web[key] = original_pyzx_x_webs_edge[
                    original_key_for_lattice_edge
                ]
        xwebs_3d.append(current_web)

for i in zwebs_3d:
    print(i)
print()
for i in xwebs_3d:
    print(i)

Finally, we can visualise the webs applied to the lattice surgery / space-time diagram produced by ***topologiq***, and compare them with the webs in the original PyZX graph.

In [ ]:
nx_g = prep_3d_g(simple_graph)
# X webs
for i, id in enumerate(boundaries):

    print(f"\nVisualising X web {i+1} of {len(boundaries)} (original v. 3D)\n")
    zx.draw(pyzx_graph, labels=True, pauli_web=xwebs[id])
    
    if edge_paths and lattice_nodes and lattice_edges:
        
        final_nx_graph, pauli_webs_graph = lattice_to_g(
            lattice_nodes, lattice_edges, nx_g, pauli_webs=xwebs_3d[i]
        )
        vis_3d_g(final_nx_graph, edge_paths, pauli_webs_graph=pauli_webs_graph)

# Z webs
for i, id in enumerate(boundaries):

    print(f"\nVisualising Z web {i+1} of {len(boundaries)} (original v. 3D)\n")
    zx.draw(pyzx_graph, labels=True, pauli_web=zwebs[id])
    
    if edge_paths and lattice_nodes and lattice_edges:
        final_nx_graph, pauli_webs_graph = lattice_to_g(
            lattice_nodes, lattice_edges, nx_g, pauli_webs=zwebs_3d[i]
        )
        vis_3d_g(final_nx_graph, edge_paths, pauli_webs_graph=pauli_webs_graph)

*Ps. We generated many visualisations. If visualisations seem sluggy or plainly do not show, run the code below to force close.*

In [ ]:
from matplotlib.pyplot import close
close()